# Notebook 04: Análisis de panel cross-country (Capítulo 6)

**Objetivo:** Replicar el análisis de panel del Capítulo 6.

**Metodología:**
- Panel balanceado: 4 economías (US, EA, UK, JP) × 96 trimestres (2000Q1–2024Q4)
- Variable dependiente: retorno trimestral del oro en moneda local
- Regresores: inflación local, tipo real local, VIX, retorno renta variable local
- Estimación: OLS pooled, Efectos Fijos (within), Efectos Aleatorios (GLS)
- Contraste de Hausman para selección del modelo preferido

**Prereq:** Ejecutar `from src.data.download import download_panel_data; download_panel_data()` o tener los CSVs en `data/raw/panel_*.csv`

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from pathlib import Path
import sys

# Añadir raíz del proyecto al path
PROJECT_ROOT = Path('..').resolve()
sys.path.insert(0, str(PROJECT_ROOT))

DATA_RAW = PROJECT_ROOT / 'data' / 'raw'
OUTPUT_TABLES = PROJECT_ROOT / 'output' / 'tables'
OUTPUT_FIGURES = PROJECT_ROOT / 'output' / 'figures'
OUTPUT_TABLES.mkdir(parents=True, exist_ok=True)
OUTPUT_FIGURES.mkdir(parents=True, exist_ok=True)

print('Setup OK')

## 1. Descarga de datos (si no están en disco)

In [ ]:
# Solo ejecutar si los CSVs no están disponibles
panel_csvs = list(DATA_RAW.glob('panel_*.csv'))
if len(panel_csvs) < 10:
    print('Descargando datos de panel...')
    from src.data.download import download_panel_data
    download_panel_data()
else:
    print(f'Datos ya disponibles: {len(panel_csvs)} archivos panel_*.csv')

## 2. Construcción del panel

In [ ]:
def load_fred(name):
    """Carga una serie FRED de panel desde data/raw/."""
    path = DATA_RAW / f'panel_fred_{name}.csv'
    s = pd.read_csv(path, index_col=0, parse_dates=True, squeeze=False)
    s.index = pd.to_datetime(s.index)
    return s.iloc[:, 0] if isinstance(s, pd.DataFrame) else s

def load_yahoo_close(name):
    """Carga precio de cierre de un ticker Yahoo de panel."""
    path = DATA_RAW / f'panel_yahoo_{name}.csv'
    df = pd.read_csv(path, index_col=0, parse_dates=True)
    df.index = pd.to_datetime(df.index)
    col = 'Close' if 'Close' in df.columns else df.columns[0]
    return df[col]

# ── Cargar series ──
# Inflación (IPC YoY)
cpi = {k: load_fred(f'cpi_{k}') for k in ['us', 'ea', 'uk', 'jp']}

# Tipos nominales 10Y
y10 = {k: load_fred(f'y10_{k}') for k in ['us', 'ea', 'uk', 'jp']}

# Tipos de cambio vs USD
fx_eurusd = load_fred('fx_eurusd')   # USD por EUR (EUR/USD directo)
fx_gbpusd = load_fred('fx_gbpusd')   # USD por GBP
fx_usdjpy = load_fred('fx_usdjpy')   # JPY por USD — invertir

# Precio del oro en USD
gold_usd = load_yahoo_close('gold_usd')

# Índices bursátiles
eq = {k: load_yahoo_close(f'eq_{k}') for k in ['us', 'ea', 'uk', 'jp']}

print('Series cargadas correctamente')

In [ ]:
# ── Precio del oro en moneda local (último día de cada mes) ──
def to_month_end(s):
    """Resample a fin de mes (último valor disponible)."""
    return s.resample('ME').last()

gold_m = to_month_end(gold_usd)
eurusd_m = to_month_end(fx_eurusd)
gbpusd_m = to_month_end(fx_gbpusd)
jpyusd_m = to_month_end(fx_usdjpy)  # JPY por USD → precio oro en JPY = gold_usd * jpyusd

gold_local = {
    'us': gold_m,
    'ea': gold_m / eurusd_m,    # Oro en EUR = USD / (EUR/USD)
    'uk': gold_m / gbpusd_m,    # Oro en GBP = USD / (GBP/USD)
    'jp': gold_m * jpyusd_m,    # Oro en JPY = USD * (JPY/USD)
}

# ── Retorno trimestral del oro en moneda local ──
# Resample a fin de trimestre, luego calcular retorno log
gold_ret_qtr = {}
for country, series in gold_local.items():
    qtr = series.resample('QE').last()
    gold_ret_qtr[country] = np.log(qtr / qtr.shift(1)) * 100  # en pp

print('Retornos trimestrales del oro calculados')
for k, v in gold_ret_qtr.items():
    print(f'  {k}: {v.dropna().shape[0]} trimestres, media = {v.mean():.2f}pp')

In [ ]:
# ── Variables explicativas (resample a trimestral) ──

def cpi_yoy(s):
    """Tasa de variación anual del IPC (%). Resampleado a trimestral (media)."""
    monthly = s.resample('ME').last()
    yoy = monthly.pct_change(12) * 100
    return yoy.resample('QE').mean()

def real_yield_qtr(y10_s, cpi_s):
    """Tipo real = tipo nominal 10Y - inflación YoY (aproximación Fisher). Trimestral."""
    y10_m = y10_s.resample('ME').last()
    cpi_m = cpi_s.resample('ME').last()
    cpi_yoy_m = cpi_m.pct_change(12) * 100
    real_m = y10_m - cpi_yoy_m
    return real_m.resample('QE').mean()

def eq_ret_qtr(eq_s):
    """Retorno trimestral del índice bursátil (log, pp)."""
    qtr = eq_s.resample('QE').last()
    return np.log(qtr / qtr.shift(1)) * 100

# VIX (común a todas las economías)
try:
    vix_raw = pd.read_csv(DATA_RAW / 'fred_VIX.csv', index_col=0, parse_dates=True).iloc[:, 0]
except FileNotFoundError:
    import yfinance as yf
    vix_raw = yf.download('^VIX', start='2000-01-01', progress=False)['Close'].iloc[:, 0]
vix_qtr = vix_raw.resample('QE').mean()

# Construir regressors por país
countries = ['us', 'ea', 'uk', 'jp']
regressors = {}
for c in countries:
    regressors[c] = pd.DataFrame({
        'inflation': cpi_yoy(cpi[c]),
        'real_yield': real_yield_qtr(y10[c], cpi[c]),
        'vix': vix_qtr,
        'eq_ret': eq_ret_qtr(eq[c]),
    })

print('Regresores trimestrales construidos')

In [ ]:
# ── Construir DataFrame de panel largo (long format) ──
frames = []
for c in countries:
    df_c = regressors[c].copy()
    df_c['gold_ret'] = gold_ret_qtr[c]
    df_c['country'] = c.upper()
    df_c.index.name = 'date'
    df_c = df_c.reset_index()
    frames.append(df_c)

panel = pd.concat(frames, ignore_index=True)

# Filtrar al período 2000Q1–2024Q4 y eliminar NaN
panel = panel[(panel['date'] >= '2000-01-01') & (panel['date'] <= '2024-12-31')]
panel = panel.dropna()

print(f'Panel: {len(panel)} observaciones | {panel["country"].nunique()} países | {panel["date"].nunique()} trimestres')
print(panel.groupby('country').size().rename('obs'))

## 3. Estadística descriptiva del panel

In [ ]:
# Estadísticas descriptivas por país
desc = panel.groupby('country')[['gold_ret', 'inflation', 'real_yield', 'vix', 'eq_ret']].describe().T
print(desc)
desc.to_csv(OUTPUT_TABLES / 'tab_6_00_panel_descriptives.csv')

## 4. Estimación: OLS pooled, Efectos Fijos, Efectos Aleatorios

In [ ]:
try:
    from linearmodels.panel import PooledOLS, PanelOLS, RandomEffects, compare
    LINEARMODELS_OK = True
except ImportError:
    print('linearmodels no instalado. Instalando...')
    import subprocess
    subprocess.run(['pip', 'install', 'linearmodels', '-q'], check=True)
    from linearmodels.panel import PooledOLS, PanelOLS, RandomEffects, compare
    LINEARMODELS_OK = True

# Preparar el índice de panel (entidad, tiempo)
panel_indexed = panel.set_index(['country', 'date'])

# Variables
y = panel_indexed['gold_ret']
X_vars = ['inflation', 'real_yield', 'vix', 'eq_ret']

import statsmodels.api as sm
X = sm.add_constant(panel_indexed[X_vars])

print('Setup para estimación OK')
print(f'Y: {y.shape}, X: {X.shape}')

In [ ]:
# ── 4.1 OLS Pooled ──
mod_pooled = PooledOLS(y, X)
res_pooled = mod_pooled.fit(cov_type='robust')
print('=== OLS POOLED ===')
print(res_pooled.summary.tables[1])

In [ ]:
# ── 4.2 Efectos Fijos (within estimator) ──
mod_fe = PanelOLS(y, X[X_vars], entity_effects=True, time_effects=False)
res_fe = mod_fe.fit(cov_type='robust')
print('=== EFECTOS FIJOS ===')
print(res_fe.summary.tables[1])

# Guardar tabla
fe_table = pd.DataFrame({
    'coef': res_fe.params,
    'std_err': res_fe.std_errors,
    'pvalue': res_fe.pvalues,
})
fe_table.to_csv(OUTPUT_TABLES / 'tab_6_01_fe_results.csv')
print('Tabla EF guardada.')

In [ ]:
# ── 4.3 Efectos Aleatorios (GLS) ──
mod_re = RandomEffects(y, X)
res_re = mod_re.fit(cov_type='robust')
print('=== EFECTOS ALEATORIOS ===')
print(res_re.summary.tables[1])

re_table = pd.DataFrame({
    'coef': res_re.params,
    'std_err': res_re.std_errors,
    'pvalue': res_re.pvalues,
})
re_table.to_csv(OUTPUT_TABLES / 'tab_6_02_re_results.csv')
print('Tabla EA guardada.')

In [ ]:
# ── 4.4 Tabla comparativa EF vs EA ──
comparison = pd.DataFrame({
    'FE_coef': res_fe.params,
    'FE_se':   res_fe.std_errors,
    'FE_pval': res_fe.pvalues,
    'RE_coef': res_re.params[X_vars],
    'RE_se':   res_re.std_errors[X_vars],
    'RE_pval': res_re.pvalues[X_vars],
}).round(4)
print('=== COMPARATIVA EF vs EA ===')
print(comparison)
comparison.to_csv(OUTPUT_TABLES / 'tab_6_03_fe_re_comparison.csv')

## 5. Test de Hausman

In [ ]:
# Test de Hausman manual: H = (b_fe - b_re)' * (V_fe - V_re)^{-1} * (b_fe - b_re)
from scipy import stats

b_fe = res_fe.params.values
b_re = res_re.params[X_vars].values
diff = b_fe - b_re

V_fe = res_fe.cov.values
V_re = res_re.cov.loc[X_vars, X_vars].values
V_diff = V_fe - V_re

# Usar pseudoinversa para mayor robustez numérica
V_diff_inv = np.linalg.pinv(V_diff)
H_stat = float(diff @ V_diff_inv @ diff)
H_df = len(X_vars)
H_pval = 1 - stats.chi2.cdf(H_stat, H_df)

hausman_result = pd.DataFrame({
    'Estadístico H': [round(H_stat, 4)],
    'Grados de libertad': [H_df],
    'p-valor': [round(H_pval, 4)],
    'Decisión': ['Efectos Fijos (rechaza H₀)' if H_pval < 0.05 else 'Efectos Aleatorios (no rechaza H₀)'],
})

print('=== TEST DE HAUSMAN ===')
print(hausman_result.to_string(index=False))
hausman_result.to_csv(OUTPUT_TABLES / 'tab_6_04_hausman.csv', index=False)

## 6. Tabla de resultados preferida y efectos fijos individuales

In [ ]:
# Modelo preferido según Hausman
preferred = 'FE' if H_pval < 0.05 else 'RE'
res_preferred = res_fe if preferred == 'FE' else res_re
print(f'Modelo preferido por Hausman: {preferred}')

# Tabla final de resultados
final_table = pd.DataFrame({
    'coef': res_preferred.params,
    'std_err': res_preferred.std_errors,
    't_stat': res_preferred.tstats,
    'pvalue': res_preferred.pvalues,
    'sig': res_preferred.pvalues.apply(
        lambda p: '***' if p < 0.01 else ('**' if p < 0.05 else ('*' if p < 0.1 else ''))
    ),
}).round(4)

print('=== TABLA FINAL DE RESULTADOS ===')
print(final_table)
final_table.to_csv(OUTPUT_TABLES / 'tab_6_05_panel_results.csv')

# Efectos fijos individuales (si FE)
if preferred == 'FE':
    fe_effects = res_fe.estimated_effects
    print('\n=== EFECTOS FIJOS INDIVIDUALES (ηᵢ) ===')
    print(fe_effects.groupby(level=0).mean().rename(columns={0: 'eta_i'}).round(4))

## 7. Visualización: coeficientes y retornos del oro por país

In [ ]:
# ── Figura: Retorno trimestral del oro en moneda local (4 países) ──
fig, axes = plt.subplots(2, 2, figsize=(14, 8), sharex=True)
country_names = {'US': 'Estados Unidos (USD)', 'EA': 'Eurozona (EUR)',
                 'UK': 'Reino Unido (GBP)', 'JP': 'Japón (JPY)'}
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

for ax, (c, color) in zip(axes.flat, zip(countries, colors)):
    ret = gold_ret_qtr[c].dropna()
    ax.bar(ret.index, ret.values, color=color, alpha=0.6, width=60)
    ax.axhline(0, color='black', linewidth=0.8)
    ax.set_title(country_names[c.upper()], fontsize=11)
    ax.set_ylabel('Retorno (pp)', fontsize=9)
    ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0f%%'))

fig.suptitle('Retorno trimestral del oro en moneda local (2000–2024)', fontsize=13)
plt.tight_layout()
plt.savefig(OUTPUT_FIGURES / 'fig_6_00_gold_returns_panel.png', dpi=150, bbox_inches='tight')
plt.show()
print('Figura guardada.')

In [ ]:
# ── Figura: Coeficientes del modelo preferido con intervalos de confianza ──
coefs = res_preferred.params
se = res_preferred.std_errors
# Solo las variables de interés (excluir const si existe)
vars_plot = [v for v in X_vars if v in coefs.index]

fig, ax = plt.subplots(figsize=(8, 4))
y_pos = np.arange(len(vars_plot))
ax.barh(y_pos, coefs[vars_plot], xerr=1.96 * se[vars_plot],
        color=['#2ca02c' if c > 0 else '#d62728' for c in coefs[vars_plot]],
        alpha=0.7, error_kw=dict(ecolor='black', capsize=4))
ax.axvline(0, color='black', linewidth=0.8)
ax.set_yticks(y_pos)
ax.set_yticklabels(['Inflación', 'Tipo real 10Y', 'VIX', 'Ret. renta variable'],
                   fontsize=10)
ax.set_xlabel('Coeficiente (pp de retorno del oro por pp de regresor)', fontsize=9)
ax.set_title(f'Coeficientes del modelo de panel ({preferred}) con IC 95%', fontsize=11)
plt.tight_layout()
plt.savefig(OUTPUT_FIGURES / 'fig_6_01_panel_coefs.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Resumen de resultados

Este notebook genera las siguientes tablas para el Capítulo 6:
- `tab_6_00_panel_descriptives.csv` — Estadística descriptiva del panel
- `tab_6_01_fe_results.csv` — Estimación por efectos fijos
- `tab_6_02_re_results.csv` — Estimación por efectos aleatorios
- `tab_6_03_fe_re_comparison.csv` — Comparativa EF vs. EA
- `tab_6_04_hausman.csv` — Test de Hausman
- `tab_6_05_panel_results.csv` — Tabla final del modelo preferido

Y las figuras:
- `fig_6_00_gold_returns_panel.png` — Retornos del oro por país
- `fig_6_01_panel_coefs.png` — Coeficientes con IC